# Embedding

In [4]:
import warnings
warnings.filterwarnings("ignore")

import os
import pickle
import cv2, spacy, numpy as np
from keras.models import model_from_json
from keras.optimizers import SGD
from sklearn.externals import joblib
from keras import backend as K
import keras
from keras.models import load_model 

In [2]:
verbose = 1

def get_image_model():
    ''' Takes the CNN weights file, and returns the VGG model update 
    with the weights. Requires the file VGG.py inside models/CNN '''
    image_model = load_model('../saved_models/resnet34.h5')

    # this is standard VGG 16 without the last two layers
    opt = SGD(lr=1e-4, momentum=0.9)
    image_model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
    #sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    # one may experiment with "adam" optimizer, but the loss function for
    # this kind of task is pretty standard
    #image_model.compile(optimizer=sgd, loss='categorical_crossentropy')
    return image_model

In [5]:
#load
vgg16_model = get_image_model()
vgg16_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_3_input (InputLayer)   (None, 256, 256, 3)       0         
_________________________________________________________________
model_3 (Model)              (None, 8, 8, 512)         21302473  
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               16777728  
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               65664     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
__________

In [6]:
def get_image_features(image_file_name):
    ''' Runs the given image_file to VGG 16 model and returns the 
    weights (filters) as a 1, 4096 dimension vector '''
    #image_features 
    image_features = np.zeros((1, 7))
    # 7> Comes from last layer of VGG Model

    # Since VGG was trained as a image of 256x256, every new image
    # is required to go through the same transformation
    im = cv2.resize(cv2.imread(image_file_name), (256, 256))
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

    im = np.reshape(im,[1,256,256,3]) 
    im = im.astype("float32")/255


   #im = im.astype(np.float32, copy=False)/255 # shape of im = (224,224,3)
    
  
    #im = np.reshape(im,[1,256,256,3])
    image_features[0,:] = vgg16_model.predict(im)[0]
    return image_features


In [7]:
PATH=os.getcwd()
data_path = PATH + '/embed-test'
data_dir_list = os.listdir(data_path)

image_features_list=[]

for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Extracting Features of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        image_features=get_image_features(data_path + '/'+ dataset + '/'+ img )
        image_features_list.append(image_features)
    
    
image_features_arr=np.asarray(image_features_list)
image_features_arr = np.rollaxis(image_features_arr,1,0)
image_features_arr = image_features_arr[0,:,:]

np.savetxt('embeddings-logs/feature_vector_test_samples.txt',image_features_arr)
#feature_vectors = np.loadtxt('feature_vectors.txt')
pickle.dump(image_features_arr, open('embeddings-logs/feature_vector_test_samples.txt.pkl', 'wb'))


Extracting Features of dataset-No_disease

Extracting Features of dataset-MDB_disease

Extracting Features of dataset-Black_Measles

Extracting Features of dataset-Blackrot

Extracting Features of dataset-mildiou

Extracting Features of dataset-Isariopsis

Extracting Features of dataset-Spyder



In [8]:
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
tf.__version__

PATH = os.getcwd()

LOG_DIR = PATH+ '/embeddings-logs'
#metadata = os.path.join(LOG_DIR, 'metadata2.tsv')

#%%
data_path = PATH + '/embed-test'
data_dir_list = os.listdir(data_path)

img_data=[]
for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
        input_img_resize=cv2.resize(input_img,(256,256))
        img_data.append(input_img_resize)
    
                
img_data = np.array(img_data)

Loaded the images of dataset-No_disease

Loaded the images of dataset-MDB_disease

Loaded the images of dataset-Black_Measles

Loaded the images of dataset-Blackrot

Loaded the images of dataset-mildiou

Loaded the images of dataset-Isariopsis

Loaded the images of dataset-Spyder



In [9]:
feature_vectors = np.loadtxt('embeddings-logs/feature_vector_test_samples.txt')
print ("feature_vectors_shape:",feature_vectors.shape)
print ("num of images:",feature_vectors.shape[0])
print ("size of individual feature vector:",feature_vectors.shape[1])

feature_vectors_shape: (70, 7)
num of images: 70
size of individual feature vector: 7


In [10]:
num_of_samples=feature_vectors.shape[0]
num_of_samples_each_class = 10

features = tf.Variable(feature_vectors, name='features')


y = np.ones((num_of_samples,),dtype='int64')

y[0:10]=0
y[10:20]=1
y[20:30]=2
y[30:40]=3
y[40:50]=4
y[50:60]=5
y[60:]=6

In [11]:
names =  ['No_disease','MDB_disease','Black_Measles','Blackrot','mildou','Isariopsis','Spyder']

metadata_file = open(os.path.join(LOG_DIR, 'metadata_7_classes.tsv'), 'w')
metadata_file.write('Class\tName\n')
k=10 # num of samples in each class
j=0

for i in range(num_of_samples):
    c = names[y[i]]
    if i%k==0:
        j=j+1
    metadata_file.write('{}\t{}\n'.format(j,c))
metadata_file.close()

In [12]:
# Taken from: https://github.com/tensorflow/tensorflow/issues/6322
def images_to_sprite(data):
    """Creates the sprite image along with any necessary padding

    Args:
      data: NxHxW[x3] tensor containing the images.

    Returns:
      data: Properly shaped HxWx3 image with any necessary padding.
    """
    if len(data.shape) == 3:
        data = np.tile(data[...,np.newaxis], (1,1,1,3))
    data = data.astype(np.float32)
    min = np.min(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1,2,3,0) - min).transpose(3,0,1,2)
    max = np.max(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1,2,3,0) / max).transpose(3,0,1,2)
    # Inverting the colors seems to look better for MNIST
    #data = 1 - data

    n = int(np.ceil(np.sqrt(data.shape[0])))
    padding = ((0, n ** 2 - data.shape[0]), (0, 0),
            (0, 0)) + ((0, 0),) * (data.ndim - 3)
    data = np.pad(data, padding, mode='constant',
            constant_values=0)
    # Tile the individual thumbnails into an image.
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3)
            + tuple(range(4, data.ndim + 1)))
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    data = (data * 255).astype(np.uint8)
    return data
#%%
sprite = images_to_sprite(img_data)
cv2.imwrite(os.path.join(LOG_DIR, 'sprite_7_classes.png'), sprite)
#scipy.misc.imsave(os.path.join(LOG_DIR, 'sprite.png'), sprite)

True

In [13]:
with tf.Session() as sess:
    saver = tf.train.Saver([features])

    sess.run(features.initializer)
    saver.save(sess, os.path.join(LOG_DIR, 'images_7_classes.ckpt'))
    
    config = projector.ProjectorConfig()
    # One can add multiple embeddings.
    embedding = config.embeddings.add()
    embedding.tensor_name = features.name
    # Link this tensor to its metadata file (e.g. labels).
    embedding.metadata_path = os.path.join(LOG_DIR, 'metadata_7_classes.tsv')
    # Comment out if you don't want sprites
    embedding.sprite.image_path = os.path.join(LOG_DIR, 'sprite_7_classes.png')
    embedding.sprite.single_image_dim.extend([img_data.shape[1], img_data.shape[1]])
    # Saves a config file that TensorBoard will read during startup.
    projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)